In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1509639,2021-07-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1509640,2021-07-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1509641,2021-07-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1509642,2021-08-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
39707,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39709,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39711,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39713,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39715,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
923035,2021-07-29,Arkansas,Arkansas,5001,2543,41.00,5000,Arkansas,AR,Arkansas,State,3017804
923037,2021-07-30,Arkansas,Arkansas,5001,2555,41.00,5000,Arkansas,AR,Arkansas,State,3017804
923039,2021-07-31,Arkansas,Arkansas,5001,2571,41.00,5000,Arkansas,AR,Arkansas,State,3017804
923041,2021-08-01,Arkansas,Arkansas,5001,2591,41.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1509639,2021-07-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1509640,2021-07-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1509641,2021-07-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1509642,2021-08-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-08-02') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
559,2021-08-02,Snohomish,Washington,42953,620.00,53061,WA,County,822083,5224.90,75.42
1116,2021-08-02,Cook,Illinois,565780,11127.00,17031,IL,County,5150233,10985.52,216.05
1672,2021-08-02,Orange,California,284534,5146.00,6059,CA,County,3175692,8959.75,162.04
2227,2021-08-02,Maricopa,Arizona,585497,10462.00,4013,AZ,County,4485414,13053.35,233.24
2782,2021-08-02,Los Angeles,California,1306400,24700.00,6037,CA,County,10039107,13013.11,246.04
...,...,...,...,...,...,...,...,...,...,...,...
1508591,2021-08-02,Wheeler,Oregon,45,1.00,41069,OR,County,1332,3378.38,75.08
1508885,2021-08-02,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1509148,2021-08-02,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00
1509407,2021-08-02,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
559,2021-08-02,Snohomish,Washington,42953,620.00,53061,WA,County,822083,5224.90,75.42,75.42,5224.90
1116,2021-08-02,Cook,Illinois,565780,11127.00,17031,IL,County,5150233,10985.52,216.05,216.05,10985.52
1672,2021-08-02,Orange,California,284534,5146.00,6059,CA,County,3175692,8959.75,162.04,162.04,8959.75
2227,2021-08-02,Maricopa,Arizona,585497,10462.00,4013,AZ,County,4485414,13053.35,233.24,233.24,13053.35
2782,2021-08-02,Los Angeles,California,1306400,24700.00,6037,CA,County,10039107,13013.11,246.04,246.04,13013.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508591,2021-08-02,Wheeler,Oregon,45,1.00,41069,OR,County,1332,3378.38,75.08,75.08,3378.38
1508885,2021-08-02,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1509148,2021-08-02,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00,0.00,4696.45
1509407,2021-08-02,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
965352,2021-08-02,Hale,Alabama,2359,78.00,1065,AL,County,14651,16101.29,532.39,532.39,16101.29,1
1051197,2021-08-02,Clarke,Alabama,3716,63.00,1025,AL,County,23622,15731.10,266.70,266.70,15731.10,2
754581,2021-08-02,Lowndes,Alabama,1458,55.00,1085,AL,County,9726,14990.75,565.49,565.49,14990.75,3
668779,2021-08-02,Etowah,Alabama,14981,370.00,1055,AL,County,102268,14648.77,361.79,361.79,14648.77,4
560856,2021-08-02,Franklin,Alabama,4590,81.00,1059,AL,County,31362,14635.55,258.27,258.27,14635.55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812001,2021-08-02,Hot Springs,Wyoming,387,3.00,56017,WY,County,4413,8769.54,67.98,67.98,8769.54,19
921074,2021-08-02,Converse,Wyoming,1154,20.00,56009,WY,County,13822,8349.01,144.70,144.70,8349.01,20
1146359,2021-08-02,Lincoln,Wyoming,1524,14.00,56023,WY,County,19830,7685.33,70.60,70.60,7685.33,21
1276855,2021-08-02,Niobrara,Wyoming,165,3.00,56027,WY,County,2356,7003.40,127.33,127.33,7003.40,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
754581,2021-08-02,Lowndes,Alabama,1458,55.00,1085,AL,County,9726,14990.75,565.49,565.49,14990.75,3,1
965352,2021-08-02,Hale,Alabama,2359,78.00,1065,AL,County,14651,16101.29,532.39,532.39,16101.29,1,2
813978,2021-08-02,Greene,Alabama,991,37.00,1063,AL,County,8111,12217.98,456.17,456.17,12217.98,32,3
320076,2021-08-02,Walker,Alabama,7817,288.00,1127,AL,County,63521,12306.17,453.39,453.39,12306.17,30,4
666795,2021-08-02,Clay,Alabama,1665,59.00,1027,AL,County,13235,12580.28,445.79,445.79,12580.28,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146359,2021-08-02,Lincoln,Wyoming,1524,14.00,56023,WY,County,19830,7685.33,70.60,70.60,7685.33,21,19
1117626,2021-08-02,Uinta,Wyoming,2430,14.00,56041,WY,County,20226,12014.24,69.22,69.22,12014.24,5,20
812001,2021-08-02,Hot Springs,Wyoming,387,3.00,56017,WY,County,4413,8769.54,67.98,67.98,8769.54,19,21
318570,2021-08-02,Teton,Wyoming,3914,11.00,56039,WY,County,23464,16680.87,46.88,46.88,16680.87,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.42,5224.90,20,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.42,5224.90,20,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.42,5224.90,20,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.42,5224.90,20,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.42,5224.90,20,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506125,2021-07-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1506126,2021-07-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1506127,2021-07-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1506128,2021-08-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
961348,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,16101.29,2,1,1.00
961349,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,16101.29,2,1,0.00
961350,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,16101.29,2,1,0.00
961351,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,16101.29,2,1,0.00
961352,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,16101.29,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165683,2021-07-29,Crook,Wyoming,475,13.00,56011,WY,County,7584,6263.19,171.41,171.41,6342.30,6,23,3.00
1165684,2021-07-30,Crook,Wyoming,476,13.00,56011,WY,County,7584,6276.37,171.41,171.41,6342.30,6,23,1.00
1165685,2021-07-31,Crook,Wyoming,476,13.00,56011,WY,County,7584,6276.37,171.41,171.41,6342.30,6,23,0.00
1165686,2021-08-01,Crook,Wyoming,476,13.00,56011,WY,County,7584,6276.37,171.41,171.41,6342.30,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
751559,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,14990.75,1,3,1.00,0.00
751560,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,14990.75,1,3,0.00,0.00
751561,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,14990.75,1,3,0.00,0.00
751562,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,14990.75,1,3,0.00,0.00
751563,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,14990.75,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749079,2021-07-29,Albany,Wyoming,4762,18.00,56001,WY,County,38880,12247.94,46.30,46.30,12314.81,23,3,26.00,0.00
749080,2021-07-30,Albany,Wyoming,4774,18.00,56001,WY,County,38880,12278.81,46.30,46.30,12314.81,23,3,12.00,0.00
749081,2021-07-31,Albany,Wyoming,4774,18.00,56001,WY,County,38880,12278.81,46.30,46.30,12314.81,23,3,0.00,0.00
749082,2021-08-01,Albany,Wyoming,4774,18.00,56001,WY,County,38880,12278.81,46.30,46.30,12314.81,23,3,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-08-02') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
382292,2021-08-02,Imperial,California,32241,742.00,6025,CA,County,181215,17791.57,409.46,409.46,17791.57,1,2,133.00,0.00
2782,2021-08-02,Los Angeles,California,1306400,24700.00,6037,CA,County,10039107,13013.11,246.04,246.04,13013.11,2,6,2361.00,9.00
189986,2021-08-02,San Bernardino,California,312200,5248.00,6071,CA,County,2180085,14320.54,240.72,240.72,14320.54,3,4,1816.00,8.00
670715,2021-08-02,Inyo,California,1473,38.00,6027,CA,County,18039,8165.64,210.65,210.65,8165.64,4,29,20.00,0.00
84777,2021-08-02,Stanislaus,California,66321,1097.00,6099,CA,County,550660,12043.91,199.22,199.22,12043.91,5,9,428.00,2.00
67405,2021-08-02,San Joaquin,California,81459,1461.00,6077,CA,County,762148,10688.08,191.70,191.70,10688.08,6,15,763.00,1.00
39192,2021-08-02,Riverside,California,317725,4667.00,6065,CA,County,2470546,12860.52,188.91,188.91,12860.52,7,7,1941.00,5.00
105174,2021-08-02,Tulare,California,60678,854.00,6107,CA,County,466195,13015.58,183.19,183.19,13015.58,8,5,191.00,0.00
607613,2021-08-02,Merced,California,33321,485.00,6047,CA,County,277680,11999.78,174.66,174.66,11999.78,9,10,0.00,0.00
38164,2021-08-02,Fresno,California,114929,1744.00,6019,CA,County,999101,11503.24,174.56,174.56,11503.24,10,12,700.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1426201,2021-08-02,Lassen,California,5888,26.00,6035,CA,County,30573,19258.82,85.04,85.04,19258.82,35,1,0.00,0.00
382292,2021-08-02,Imperial,California,32241,742.00,6025,CA,County,181215,17791.57,409.46,409.46,17791.57,1,2,133.00,0.00
813920,2021-08-02,Kings,California,24970,251.00,6031,CA,County,152940,16326.66,164.12,164.12,16326.66,11,3,152.00,0.00
189986,2021-08-02,San Bernardino,California,312200,5248.00,6071,CA,County,2180085,14320.54,240.72,240.72,14320.54,3,4,1816.00,8.00
105174,2021-08-02,Tulare,California,60678,854.00,6107,CA,County,466195,13015.58,183.19,183.19,13015.58,8,5,191.00,0.00
2782,2021-08-02,Los Angeles,California,1306400,24700.00,6037,CA,County,10039107,13013.11,246.04,246.04,13013.11,2,6,2361.00,9.00
39192,2021-08-02,Riverside,California,317725,4667.00,6065,CA,County,2470546,12860.52,188.91,188.91,12860.52,7,7,1941.00,5.00
239013,2021-08-02,Kern,California,113810,1426.00,6029,CA,County,900202,12642.72,158.41,158.41,12642.72,13,8,594.00,1.00
84777,2021-08-02,Stanislaus,California,66321,1097.00,6099,CA,County,550660,12043.91,199.22,199.22,12043.91,5,9,428.00,2.00
607613,2021-08-02,Merced,California,33321,485.00,6047,CA,County,277680,11999.78,174.66,174.66,11999.78,9,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
382292,2021-08-02,Imperial,California,32241,742.00,6025,CA,County,181215,17791.57,409.46,409.46,17791.57,1,2,133.00,0.00
2782,2021-08-02,Los Angeles,California,1306400,24700.00,6037,CA,County,10039107,13013.11,246.04,246.04,13013.11,2,6,2361.00,9.00
189986,2021-08-02,San Bernardino,California,312200,5248.00,6071,CA,County,2180085,14320.54,240.72,240.72,14320.54,3,4,1816.00,8.00
670715,2021-08-02,Inyo,California,1473,38.00,6027,CA,County,18039,8165.64,210.65,210.65,8165.64,4,29,20.00,0.00
84777,2021-08-02,Stanislaus,California,66321,1097.00,6099,CA,County,550660,12043.91,199.22,199.22,12043.91,5,9,428.00,2.00
67405,2021-08-02,San Joaquin,California,81459,1461.00,6077,CA,County,762148,10688.08,191.70,191.70,10688.08,6,15,763.00,1.00
39192,2021-08-02,Riverside,California,317725,4667.00,6065,CA,County,2470546,12860.52,188.91,188.91,12860.52,7,7,1941.00,5.00
105174,2021-08-02,Tulare,California,60678,854.00,6107,CA,County,466195,13015.58,183.19,183.19,13015.58,8,5,191.00,0.00
607613,2021-08-02,Merced,California,33321,485.00,6047,CA,County,277680,11999.78,174.66,174.66,11999.78,9,10,0.00,0.00
38164,2021-08-02,Fresno,California,114929,1744.00,6019,CA,County,999101,11503.24,174.56,174.56,11503.24,10,12,700.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5022,07/29/21,Lassen,5876,26.00,19219.57,85.04,35,1,2.00,0.00
5023,07/30/21,Lassen,5881,26.00,19235.93,85.04,35,1,5.00,0.00
5024,07/31/21,Lassen,5888,26.00,19258.82,85.04,35,1,7.00,0.00
5025,08/01/21,Lassen,5888,26.00,19258.82,85.04,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5108,07/29/21,Fresno,114048,1744.00,11415.06,174.56,10,12,284.00,-2.00
5109,07/30/21,Fresno,114229,1744.00,11433.18,174.56,10,12,181.00,0.00
5110,07/31/21,Fresno,114229,1744.00,11433.18,174.56,10,12,0.00,0.00
5111,08/01/21,Fresno,114229,1744.00,11433.18,174.56,10,12,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)